In [3]:
import ee

ee.Authenticate()

ee.Initialize(project='ee-caiosimplicioarantes') # Replace with your project ID

from ee_s1_ard import S1ARDImageCollection

# Define input parameters
geometry = ee.Geometry.Polygon(
    [[[5.0, 50.0], [5.5, 50.0], [5.5, 50.5], [5.0, 50.5], [5.0, 50.0]]]
)
start_date = '2021-01-01'
stop_date = '2021-12-31'

processor = S1ARDImageCollection(
    geometry=geometry,
    start_date=start_date,
    stop_date=stop_date,
    polarization="VVVH",
    apply_border_noise_correction=True,
    apply_terrain_flattening=True,
    apply_speckle_filtering=True,
    output_format="DB"
)

collection = processor.get_collection()

In [4]:
collection_size = collection.size().getInfo()

print(f"Number of images in the collection: {collection_size}")

Number of images in the collection: 148


In [5]:
from datetime import datetime

# Get the list of image dates in the collection
dates = collection.aggregate_array('system:time_start').getInfo()

# Convert timestamps to readable date strings
date_strings = [datetime.utcfromtimestamp(ts / 1000).strftime('%Y-%m-%d') for ts in dates]
print(date_strings)

['2021-01-07', '2021-01-07', '2021-01-12', '2021-01-19', '2021-01-19', '2021-01-24', '2021-01-31', '2021-01-31', '2021-02-05', '2021-02-12', '2021-02-12', '2021-02-17', '2021-02-24', '2021-02-24', '2021-03-01', '2021-03-08', '2021-03-08', '2021-03-13', '2021-03-20', '2021-03-20', '2021-03-25', '2021-04-01', '2021-04-01', '2021-04-06', '2021-04-13', '2021-04-13', '2021-04-18', '2021-04-25', '2021-04-25', '2021-04-30', '2021-05-07', '2021-05-07', '2021-05-12', '2021-05-19', '2021-05-19', '2021-05-31', '2021-05-31', '2021-06-05', '2021-06-12', '2021-06-12', '2021-06-17', '2021-06-24', '2021-06-24', '2021-06-29', '2021-07-06', '2021-07-06', '2021-07-11', '2021-07-18', '2021-07-18', '2021-07-23', '2021-07-30', '2021-07-30', '2021-08-04', '2021-08-11', '2021-08-11', '2021-08-16', '2021-08-23', '2021-08-23', '2021-08-28', '2021-09-04', '2021-09-04', '2021-09-09', '2021-09-16', '2021-09-16', '2021-09-21', '2021-09-28', '2021-09-28', '2021-10-03', '2021-10-10', '2021-10-10', '2021-10-15', '2021

C:\Users\pc\AppData\Local\Temp\ipykernel_20960\2530563381.py:7: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  date_strings = [datetime.utcfromtimestamp(ts / 1000).strftime('%Y-%m-%d') for ts in dates]


In [ ]:
# Get mean VV and VH backscatter over the geometry for each image in the collection
def extract_mean_backscatter(image):
    stats = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geometry,
        scale=10,
        maxPixels=1e9
    )
    return ee.Feature(None, {
        'system:time_start': image.get('system:time_start'),
        'VV': stats.get('VV'),
        'VH': stats.get('VH')
    })

# Map the function over the collection and get the results as a list of features
features = collection.map(extract_mean_backscatter).getInfo()['features']

# Extract time, VV, and VH values
timeseries = []
for f in features:
    props = f['properties']
    timeseries.append({
        'date': datetime.utcfromtimestamp(props['system:time_start'] / 1000).strftime('%Y-%m-%d'),
        'VV': props['VV'],
        'VH': props['VH']
    })

# Print the first 5 entries of the timeseries
print(timeseries[:5])